<a href="https://colab.research.google.com/github/AuraFrizzati/NHSW-NHFD-Web-Scraper/blob/main/V2_GoogleColab_NHFD_WebScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello world")

hello world
